In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob

import cudf
import numpy as np
import pandas as pd

import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

In [13]:
import os
import pandas as pd
from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
from merlin.schema import Schema
from merlin.io import Dataset

import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.schema import Schema, Tags

In [15]:
DATA_DIR =os.getcwd()

In [60]:
# MOVIELENS_PATH = os.path.join(DATA_DIR, 'ml-100k')
# os.makedirs(DATA_DIR, exist_ok=True)

# DATA_FILE = os.path.join(MOVIELENS_PATH, 'processed_ml-100k.parquet')
df = pd.read_csv("netflix_sampled_logs.csv", names=['user_id', 'item_id', 'rating', 'timestamp'])

df['user_id'] = df['user_id'].astype(str)
df['item_id'] = df['item_id'].astype(str)
df.to_parquet(DATA_FILE, index=False)

df.sort_values(by=["user_id", "timestamp"], inplace=True)
df.reset_index(inplace=True, drop=True)

In [61]:
df.shape

(116410, 4)

In [72]:
df=df.sample(20000)

In [86]:
a=df.copy()

In [73]:
df["rating"]=df["rating"].astype(float)
df["rating"]=df["rating"].astype(int)
df["timestamp"]=df["timestamp"].astype(int)

In [82]:
v1=pd.read_csv("netflix_v1_df.csv")

In [83]:
v1['user_id'] = v1['user_id'].astype(str)
v1['item_id'] = v1['item_id'].astype(str)

In [85]:
v1["rating"]=v1["rating"].astype(float)
v1["rating"]=v1["rating"].astype(int)

In [87]:
a.shape

(25207, 4)

In [88]:
df=pd.concat([df, v1])

In [89]:
a.shape

(25207, 4)

In [90]:
SESSIONS_MAX_LENGTH =10
categ_feats = ['item_id'] >> nvt.ops.Categorify()

groupby_feats = categ_feats + ['user_id', 'rating', 'timestamp']
# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["user_id"], 
    aggs={
        "item_id": ["list", "count"],
        "rating": ["list"],
        "timestamp": ["list"]
        },
    name_sep="-")

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
)  

MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'user_id'] + 
    sequence_features_truncated_item + 
    groupby_features['rating-list', 'timestamp-list']  
)

filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)
seq_feats_list = (
    filtered_sessions['item_id-list', 'rating-list', 'timestamp-list'] 
    >> nvt.ops.ValueCount() 
)

workflow = nvt.Workflow(filtered_sessions['user_id'] + seq_feats_list)
dataset = nvt.Dataset(df)

In [91]:
INPUT_DATA_DIR=os.getcwd()
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "movielens-nvt"))
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow_etl"))
sessions_gdf = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "movielens-nvt/part_0.parquet"))

In [92]:
from transformers4rec.utils.data_utils import save_time_based_splits

In [93]:
sessions_gdf.shape

(5955, 4)

In [ ]:
OUTPUT_DIR = os.path.join(INPUT_DATA_DIR, "time_based_splits")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 세션 데이터를 NVT Dataset으로 로드
sessions_dataset = nvt.Dataset(sessions_gdf)

# 세션 데이터를 user_id 기반으로 시간 분할
save_time_based_splits(
    data=sessions_dataset,
    output_dir=OUTPUT_DIR,
    partition_col='user_id',  # user_id를 기준으로 분할
    timestamp_col='timestamp-list',  # 시간 열
)

print(f"Time-based splits saved in {OUTPUT_DIR}")

Creating time-based splits:   6%|███▏                                                | 365/5955 [00:10<02:29, 37.48it/s]